In [1]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm
from dotenv import load_dotenv
import os

In [2]:
# Caminhos dos arquivos
CAMINHO_CSV = "../terceirizados/df_contratos.csv"
CAMINHO_SAIDA = "../data/terceirizados/arquivos_tratados_contratos_gov/consolidado_validacao_completo.csv"

In [3]:
# Carregar o CSV original como string para preservar formatações
df = pd.read_csv(CAMINHO_CSV, dtype="str")

# Padronizar os campos utilizados na busca
df['nr_contrato'] = df['nr_contrato'].astype(str).str.strip()
df['nr_orgao'] = df['nr_orgao'].astype(str).str.strip()
df['cnpj'] = df['cnpj'].astype(str).str.strip()

In [4]:
# Carregar validações anteriores, se existirem
try:
    df_validado = pd.read_csv(CAMINHO_SAIDA, dtype=str)
    for coluna in [
        "id_contrato", "status_validacao", "objeto_contrato", "objeto_compra",
        "dataAssinatura", "dataPublicacaoDOU", "dataInicioVigencia", "dataFimVigencia",
        "tipo_fornecedor", "valorInicialCompra", "valorFinalCompra"
    ]:
        if coluna not in df.columns:
            df[coluna] = None
        if coluna in df_validado.columns:
            df[coluna] = df_validado[coluna]
except FileNotFoundError:
    for coluna in [
        "id_contrato", "status_validacao", "objeto_contrato", "objeto_compra",
        "dataAssinatura", "dataPublicacaoDOU", "dataInicioVigencia", "dataFimVigencia",
        "tipo_fornecedor", "valorInicialCompra", "valorFinalCompra"
    ]:
        df[coluna] = None

In [5]:
load_dotenv()
API_KEY = os.getenv("API_PORTAL_TRANSPARENCIA")

In [6]:
# 🔍 Filtrar apenas contratos ainda não validados
df_nao_validados = df[
    df['id_contrato'].isna() & (df['status_validacao'] != 'Não encontrado')
].copy()

print(f"🔎 Serão validadas {len(df_nao_validados)} linhas...")

# Sessão com headers da API
session = requests.Session()
base_url = "https://api.portaldatransparencia.gov.br/api-de-dados/contratos"
headers = {
    "chave-api-dados": API_KEY,
    "Accept": "application/json"
}
session.headers.update(headers)

🔎 Serão validadas 3095 linhas...


In [7]:
def respeitar_limite_requisicoes():
    """Pausa entre requisições para evitar sobrecarga"""
    time.sleep(random.uniform(0.5, 1.5))

In [8]:
def buscar_dados_contrato(nr_contrato, nr_orgao, cnpj):
    """Faz até duas tentativas: com e sem zeros à esquerda no número do contrato"""
    tentativas = [nr_contrato, nr_contrato.lstrip("0")]

    for tentativa_contrato in tentativas:
        try:
            respeitar_limite_requisicoes()
            url = f"{base_url}?numero={tentativa_contrato}&codigoOrgao={nr_orgao}"
            response = session.get(url, timeout=10)

            if response.status_code != 200:
                continue

            try:
                data = response.json()
            except ValueError:
                continue

            if not data:
                continue

            if len(data) == 1:
                contrato = data[0]
            else:
                contrato = next(
                    (contrato for contrato in data
                     if contrato['fornecedor'].get('cnpjFormatado', '').strip() == cnpj),
                    None
                )
                if not contrato:
                    continue

            return {
                "id_contrato": contrato.get("id", ""),
                "status_validacao": "OK",
                "objeto_contrato": contrato.get("objeto", ""),
                "objeto_compra": contrato.get("compra", {}).get("objeto", ""),
                "dataAssinatura": contrato.get("dataAssinatura", ""),
                "dataPublicacaoDOU": contrato.get("dataPublicacaoDOU", ""),
                "dataInicioVigencia": contrato.get("dataInicioVigencia", ""),
                "dataFimVigencia": contrato.get("dataFimVigencia", ""),
                "tipo_fornecedor": contrato.get("fornecedor", {}).get("tipo", ""),
                "valorInicialCompra": contrato.get("valorInicialCompra", ""),
                "valorFinalCompra": contrato.get("valorFinalCompra", "")
            }

        except requests.exceptions.RequestException:
            continue

    return {"status_validacao": "Não encontrado"}

In [9]:
# 🚀 Loop principal
print("🚧 Iniciando processo de validação incremental...\n")
for idx, row in tqdm(df_nao_validados.iterrows(), total=len(df_nao_validados)):
    resultado = buscar_dados_contrato(
        row['nr_contrato'], row['nr_orgao'], row['cnpj']
    )
    for chave, valor in resultado.items():
        df.at[idx, chave] = valor

    # Salvamento incremental
    df.to_csv(CAMINHO_SAIDA, index=False)

print("\n✅ Validação finalizada com sucesso e arquivo salvo!")

🚧 Iniciando processo de validação incremental...



100%|█████████████████████████████████████| 3095/3095 [1:55:11<00:00,  2.23s/it]


✅ Validação finalizada com sucesso e arquivo salvo!
